In [ ]:
from matplotlib import pylab 
params = {'xtick.labelsize': 18, 
'ytick.labelsize': 18, 
'axes.titlesize' : 22, 
'axes.labelsize' : 20, 
'legend.fontsize': 18, 
'legend.title_fontsize': 22, 
'figure.titlesize': 24 } 
pylab.rcParams.update(params)
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from pprint import pprint
from prepare import prepare_data
from sklearn.model_selection import train_test_split


In [ ]:
df = pd.read_csv("data_HW3.csv")
ASSAF_ID = 207376807
DANIEL_ID = 209088723
randomness = ASSAF_ID % 100 + DANIEL_ID % 100
train_set, test_set = train_test_split(
    df, random_state=randomness, test_size=0.2
)
prepared_train_set = prepare_data(df, train_set)
prepared_test_set = prepare_data(df, test_set)

# Preliminary: Updated Data Loading

# Section 1

## Q123

In [ ]:
from typing import List

class Datapoint:
    pass

class Label:
    pass

def get_mean(X,Y):
    pass

def get_variance(X,Y):
    pass

def get_huber_delta(X: List[Datapoint], Y: List[Label]):
    variance = get_variance(X, Y)
    delta_candidates = [0.1 * variance, 0.1 * variance]
    return 2*variance
    
    

## Q124

## Q125

# Section 2

## Q234

## Q235

# Section 3

## Q345

## Q346

## Q347

# IML Major HW 3 - Linear Regression
| Name | ID |
|------|----|
| Daniel Gershkovich   | 209088723  |
| Assaf Alon   | 207376807  |

In [ ]:
df = pd.read_csv("data_HW3.csv")
ASSAF_ID = 207376807
DANIEL_ID = 209088723
randomness = ASSAF_ID % 100 + DANIEL_ID % 100
train_set, test_set = train_test_split(
    df, random_state=randomness, test_size=0.2
)
prepared_train_set = prepare_data(train_set, train_set)
prepared_test_set = prepare_data(train_set, test_set)